This tool is built to create reports on any topic requested using the climate tracker database

# Setup

In [8]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
import json
from dotenv import load_dotenv
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema import BaseOutputParser
import re

load_dotenv()

True

In [9]:
# Load models
super_basic_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Llama-3.2-1B-Instruct",
    temperature=0
)

standard_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0
)

larger_context_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0
)

#### 1.1 Prompts

In [ ]:
# For the template generating LLM
template_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to create a template for the structure of a 1 page report on {topic}. 
     
     Here are some example templates to base your response on:
        ##METADATA 

        *Country: 

        ##REPORT 

        Q1: What are the relevant background information for indicator X, Y or Z? 
        … 
        Question N (CHALLENGING): How has the legislation of this country changed in the past 50 years? 
        
     
        ##TOPIC: Just transition 

        *Countries with relevant information: … 

        ##REPORT 

        Q1: Which countries have made plans for a just energetic transition? 

        A: Country A [citation], B [citation] and C [citation] have a similar law. They all promise X, Y and Z. On the other hand, the following countries … differ because … 
        A: … 
        
        A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
        OR a national long-term decarbonisation target.

    Respond with only the template of the {topic} report and nothing else."""),
    ("human", "Context: {topic}")
])

# Seperate template into sub-sections USE LOW POWER LMMM
section_seperator_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to extract the subsections from {template}.         
    Respond with only the template subsections nothing else."""),
# SPECIFY HOW TO STRUCUTRE SUBSECTIONS SO CAN BE EXTRACTED EASILY
    ("human", "{template}")
])

# For the hypothetical response generating LLM
hypothetical_response_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to generate a hypothetical response for the following question: {subsection}. 
    The response should be based on the template {template}, a template for a report on topic {topic}.
    Respond with only the hypothetical response nothing else."""),
    ("human", "{subsection}, {template}, {topic}")
])
# Prompts for each of the sub-section models
subsection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to create a one paragraph {subsection} as part of a report on {topic}.
    Use only information on {context} to create the paragraph.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    The paragraph should be concise and informative, summarizing the key points relevant to the subsection.     
    Respond with only the paragraph for that subsections nothing else."""),
    ("human", "{subsection}, {context}, {topic}")
])


# THIS COULD BE DONE WITHOUT USING AN LLM???
# Prompt for the compling model
compiling_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to compile the following subsections {all_subsections}.
    The report should be structured according to the template {template}.
    The report must match exactly the template structure.
    Respond with only the compiled report nothing else."""),
    ("human", "{all_subsections}, {template}")
])

# Prompt for the checking model
checking_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to check the following subsections {all_subsections} for consistency and completeness.
    Also ensure that the report {report} matches the template {template}.
    Also ensure that the report is related to the topic {topic}.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    If 
        1) One of the subsections appears incorrect or incomplete → output ONLY: (**subsection name**), incomplete
        2) The report does not match the template → output ONLY: not match
        3) The report is not related to the topic → output ONLY: not related
        4) Everything is correct → output ONLY: ok
     
    Respond with only the result of the check (ok, not related, not match, (**subsection name**) incomplete) nothing else.
    """),
    ("human", "{all_subsections}, {template}, {report}, {topic}, {context}")
])

# Prompt for the rewrite subsection model
rewrite_subsection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to rewrite the following subsection {subsection} to make it more complete and consistent.
    Use the template {template} as a guide for the structure.
    The rewritten subsection should be concise and informative, summarizing the key points relevant to the subsection.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    Respond with only the rewritten subsection nothing else."""),
    ("human", "{subsection}, {template}, {context}")
])

### 1.2 Prompt Chaining

Workflow 
1. Generate initial template (med model)

2. Human template approval (if fail rewrite with higher power model) 

3. Extract subsections from approved template (low model)

4. Generate hypothetical responses for each subsection (med model) 
    - Retrieve relevant context using hypothetical responses

6. Generate actual subsections using retrieved context (high model)

7. Compile subsections into full report (low model)

8. Quality check (structure, completeness, relevance - if any fail rewrite with higher model) (high model)

9. Human final approval with specific feedback collection (high model)

In [11]:
class ReportWorkflow:
    def __init__(self, low_model, med_model, high_model, retriever=None):
        self.low_model = low_model
        self.med_model = med_model
        self.high_model = high_model
        self.retriever = retriever
        
    def _extract_subsections(self, template):
        """Extract subsections from template - can be done without LLM"""
        # Simple regex-based extraction (replace with LLM if needed)
        sections = re.findall(r'Q\d+:.*?(?=Q\d+:|$)', template, re.DOTALL)
        return [section.strip() for section in sections if section.strip()]
    
    def _human_approval(self, content, prompt_msg):
        """Simulate human approval - replace with actual UI"""
        print(f"\n{prompt_msg}")
        print(content)
        return input("Approve? (y/n): ").lower() == 'y'
    
    def _retrieve_context(self, hypothetical_response):
        """Retrieve relevant context using hypothetical response"""
        if self.retriever:
            return self.retriever.get_relevant_documents(hypothetical_response)
        return "No context available"  # Placeholder
    
    def generate_template(self, topic):
        """Step 1-2: Generate and approve template"""
        chain = template_prompt | self.med_model
        template = chain.invoke({"topic": topic}).content
        
        if not self._human_approval(template, "Review template:"):
            # Retry with higher power model
            chain = template_prompt | self.high_model
            template = chain.invoke({"topic": topic}).content
            
        return template
    
    def extract_subsections(self, template):
        """Step 3: Extract subsections"""
        # Using simple extraction - uncomment below for LLM approach
        return self._extract_subsections(template)
        
        # LLM approach:
        # chain = section_seperator_prompt | self.low_model
        # result = chain.invoke({"template": template}).content
        # return result.split('\n')
    
    def generate_subsection_content(self, subsection, template, topic):
        """Step 4-6: Generate hypothetical response, retrieve context, create actual subsection"""
        # Generate hypothetical response
        hyp_chain = hypothetical_response_prompt | self.med_model
        hypothetical = hyp_chain.invoke({
            "subsection": subsection,
            "template": template,
            "topic": topic
        }).content
        
        # Retrieve context
        context = self._retrieve_context(hypothetical)
        
        # Generate actual subsection
        sub_chain = subsection_prompt | self.high_model
        return sub_chain.invoke({
            "subsection": subsection,
            "context": context,
            "topic": topic
        }).content
    
    def compile_report(self, subsections, template):
        """Step 7: Compile subsections into report"""
        chain = compiling_prompt | self.low_model
        return chain.invoke({
            "all_subsections": "\n\n".join(subsections),
            "template": template
        }).content
    
    def quality_check_and_fix(self, subsections, template, report, topic):
        """Step 8: Quality check with potential fixes"""
        check_chain = checking_prompt | self.high_model
        
        max_retries = 3
        for attempt in range(max_retries):
            check_result = check_chain.invoke({
                "all_subsections": "\n\n".join(subsections),
                "template": template,
                "report": report,
                "topic": topic
            }).content.strip()
            
            if check_result == "ok":
                return report, subsections
            
            # Handle different failure cases
            if "incomplete" in check_result:
                # Extract subsection name and rewrite
                section_match = re.search(r'\*\*(.*?)\*\*', check_result)
                if section_match:
                    section_name = section_match.group(1)
                    # Find and rewrite the problematic subsection
                    for i, subsection in enumerate(subsections):
                        if section_name.lower() in subsection.lower():
                            rewrite_chain = rewrite_subsection_prompt | self.high_model
                            subsections[i] = rewrite_chain.invoke({
                                "subsection": subsection,
                                "template": template
                            }).content
                            break
            
            elif check_result in ["not match", "not related"]:
                # Regenerate entire report
                report = self.compile_report(subsections, template)
        
        return report, subsections
    
    def run_workflow(self, topic):
        """Main workflow execution"""
        print(f"Starting report generation for topic: {topic}")
        
        # Step 1-2: Generate and approve template
        template = self.generate_template(topic)
        
        # Step 3: Extract subsections
        subsections_list = self.extract_subsections(template)
        
        # Step 4-6: Generate content for each subsection
        subsection_contents = []
        for subsection in subsections_list:
            content = self.generate_subsection_content(subsection, template, topic)
            subsection_contents.append(content)
        
        # Step 7: Compile report
        report = self.compile_report(subsection_contents, template)
        
        # Step 8: Quality check and fix
        final_report, final_subsections = self.quality_check_and_fix(
            subsection_contents, template, report, topic
        )
        
        # Step 9: Human final approval
        if self._human_approval(final_report, "Final report review:"):
            print("Report approved and completed!")
            return final_report
        else:
            feedback = input("Please provide feedback: ")
            print(f"Report rejected. Feedback: {feedback}")
            return None

# Usage
workflow = ReportWorkflow(
    low_model=super_basic_model,
    med_model=standard_model, 
    high_model=larger_context_model,
    retriever=None  # Add your retriever here
)

### testing


In [12]:
cp1a_1_content = [
  {
    "chunk_content": "The UK’s 2035 Nationally Determined Contribution (NDC) target, announced by the Prime Minister at COP29 in November 2024, is to reduce all greenhouse gas emissions by at least 81% on 1990 levels, excluding international aviation and shipping emissions.",
    "document": "UK’s 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The UK’s 2035 Updated Nationally Determined Contribution confirms the topline ambition of reducing emissions by 81% (based on 1990 levels). This target is in line with advice given by the Climate Change Committee in October 2024, and represents a 1.5°C-aligned, economy-wide absolute emissions reduction target.",
    "document": "The UK's NDC and the Seventh Carbon Budget - Chapter Zero",
    "page_number": 1
  },
  {
    "chunk_content": "The UK government has been advised by the independent Climate Change Committee (CCC) to commit to reducing greenhouse gas emissions by 81% by 2035 compared to 1990 levels. This recommendation aims to be both fair and ambitious in contributing to the Paris Agreement.",
    "document": "UK must raise its climate ambition to restore global reputation, says adviser - Financial Times",
    "page_number": 1
  },
  {
    "chunk_content": "At COP29 in Baku, British PM Keir Starmer announced an ambitious target to reduce the UK's greenhouse gas emissions by at least 81% by 2035, compared to 1990 levels. This is an increase from the previous target of a 68% cut by 2030, with the ultimate goal of achieving carbon neutrality by 2050.",
    "document": "COP29: British PM Starmer wants to regain 'climate leadership' - Le Monde",
    "page_number": 1
  },
  {
    "chunk_content": "The UK submitted its 2035 NDC to the UNFCCC on 30 January 2025. It will also be available on the UNFCCC NDC registry. The UK will produce an updated cross-economy plan in due course, outlining the policies needed to deliver carbon budgets 4-6 and the 2030 and 2035 NDCs on the pathway to net zero by 2050.",
    "document": "UK's 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK",
    "page_number": 1
  }
]

cp1a_2_content = [
  {
    "chunk_content": "The Climate Change Act 2008 was the first law of its kind. It makes long-term emissions reduction targets legally-binding, opening the door to citizen lawsuits if the government misses them, and introduces interim targets or 'carbon budgets' to keep the UK on track.",
    "document": "Climate Legislation in the United Kingdom - Canadian Climate Institute",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act, as amended in 2019, commits the UK to ‘net zero’ by 2050. The original act, passed in 2008, committed the UK to an 80% reduction of greenhouse gas emissions by 2050, compared to 1990 levels.",
    "document": "Climate change targets: the road to net zero? - House of Lords Library",
    "page_number": 1
  },
  {
    "chunk_content": "In June 2021, the government set in law the sixth carbon budget (CB6) limiting the volume of greenhouse gases emitted from 2033 to 2037. CB6 reduces emissions by approximately 78% by 2035 compared to 1990 levels.",
    "document": "Why Net Zero - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The UK Parliament passed The Climate Change Act 2008 (2050 Target Amendment) Order 2019, which amends Section 1 of the Climate Change Act 2008 to require the UK to reduce its GHG emissions by 100% compared to their level in 1990.",
    "document": "UK Becomes First G7 Country to Legislate for 'Net-Zero' Emissions - WilmerHale",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act (2008) made the UK the first country to establish a long-term legally binding framework to cut carbon emissions. It contains a target requiring emissions reductions to Net Zero by 2050.",
    "document": "Climate action - Climate Change Committee",
    "page_number": 1
  }
]

cp1a_3_content = [
  {
    "chunk_content": "The UK’s Net Zero Strategy outlines a comprehensive plan to achieve net zero greenhouse gas emissions by 2050, setting legally binding carbon budgets and detailing sector-specific policies to reduce emissions.",
    "document": "Net Zero Strategy: Build Back Greener - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act 2008 mandates the UK government to set legally binding carbon budgets, which cap the amount of greenhouse gases emitted over a five-year period, ensuring progress towards the 2050 net zero target.",
    "document": "Climate Change Act 2008 - legislation.gov.uk",
    "page_number": 1
  },
  {
    "chunk_content": "The UK Emissions Trading Scheme (UK ETS), established in 2021, sets a cap on total greenhouse gas emissions and allows trading of emission allowances, providing a market-based approach to incentivize emission reductions.",
    "document": "UK Emissions Trading Scheme - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Renewables Obligation requires electricity suppliers to source an increasing proportion of electricity from renewable sources, supporting the development of renewable energy technologies through financial incentives.",
    "document": "Renewables Obligation (United Kingdom) - Wikipedia",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Levy is a tax on energy delivered to non-domestic users in the UK, designed to encourage energy efficiency and reduce carbon emissions in the industrial, commercial, and public services sectors.",
    "document": "Climate Change Levy - GOV.UK",
    "page_number": 1
  }
]

cp1a_4_content = [
  {
    "chunk_content": "The Environment Act 2021 provides a broad framework through which specific environmental rules and regulation will operate. While it remains to be seen how the government will implement the Act’s provisions, it represents a significant opportunity for environmental law to drive the UK’s transition to net zero. The Act requires the Government to devise long-term legally binding targets concerning biodiversity, water, air quality, resource efficiency and waste management.",
    "document": "Environmental Law and Climate Change - Centre for Climate Engagement",
    "page_number": 1
  },
  {
    "chunk_content": "The Government’s 25-year environmental plan supports the Act by requiring that all policies, programmes, and investment decisions take due consideration of climate change.",
    "document": "Environmental Law and Climate Change - Centre for Climate Engagement",
    "page_number": 1
  },
  {
    "chunk_content": "The Environment Act 2021, passed in the UK, sets forth new environmental targets and establishes the Office for Environmental Protection (OEP) as an oversight body. It outlines a governance framework and mandates an environmental improvement plan for England and Wales, focusing on enhancing environmental standards and biodiversity.",
    "document": "British environmental law - Wikipedia",
    "page_number": 1
  },
  {
    "chunk_content": "The UK’s Net Zero Strategy outlines a comprehensive plan to achieve net zero greenhouse gas emissions by 2050, setting legally binding carbon budgets and detailing sector-specific policies to reduce emissions.",
    "document": "Net Zero Strategy: Build Back Greener - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Net Zero Strategy builds on the Government’s 10 point plan for a green industrial revolution which was published on 18 November 2020. It focuses on advancing offshore wind, driving the growth of low carbon hydrogen, delivering new and advanced nuclear power, accelerating the shift to zero emission vehicles, and other key areas.",
    "document": "Government support for a nature and climate declaration - House of Commons Library",
    "page_number": 1
  }
]

cp1a_retrieved_chunks = [cp1a_1_content, cp1a_2_content, cp1a_3_content, cp1a_4_content]


In [ ]:
final_report = workflow.run_workflow("Does the country have a framework climate law or equivalent")

In [ ]:
# Create workflow normally
workflow = ReportWorkflow(
    low_model=super_basic_model,
    med_model=standard_model, 
    high_model=larger_context_model,
    retriever=None
)

# Temporarily override the _retrieve_context method
def temp_retrieve_context(self, hypothetical_response):
    return cp1a_retrieved_chunks

# Monkey patch the method
workflow._retrieve_context = temp_retrieve_context.__get__(workflow, ReportWorkflow)



In [14]:
# Run the workflow with your modified retriever
final_report = workflow.run_workflow("Does the country have a framework climate law or equivalent")

Starting report generation for topic: Does the country have a framework climate law or equivalent

Review template:
##METADATA 

*Country: 

##REPORT 

Q1: Does the country have a framework climate law or equivalent? 
    A: Yes, the country has a framework climate law or equivalent (page X, doc Y) [citation]. 
    OR 
    A: No, the country does not have a framework climate law or equivalent (page X, doc Y) [citation]. 

Q2: What are the key provisions of the framework climate law or equivalent? 
    A: The law includes provisions for X, Y, and Z (page X, doc Y) [citation]. 

Q3: Is the framework climate law or equivalent aligned with the Paris Agreement? 
    A: Yes, the law is aligned with the Paris Agreement (page X, doc Y) [citation]. 
    OR 
    A: No, the law is not aligned with the Paris Agreement (page X, doc Y) [citation]. 

Q4: Are there any notable gaps or weaknesses in the framework climate law or equivalent? 
    A: The law lacks provisions for X, Y, and Z (page X, doc Y